In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Check the GPU
!nvidia-smi

Tue Jan 25 19:26:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Clone the repo
!git clone https://github.com/justinline/stylegan2-ada
!pip install -e ./stylegan2-ada


Cloning into 'stylegan2-ada'...
remote: Enumerating objects: 407, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 407 (delta 17), reused 36 (delta 10), pack-reused 364
Receiving objects: 100% (407/407), 56.17 MiB | 13.74 MiB/s, done.
Resolving deltas: 100% (217/217), done.
Obtaining file:///content/stylegan2-ada
     |████████████████████████████████| 21.8 MB 1.3 MB/s 
     |████████████████████████████████| 15.7 MB 69.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Running setup.py develop for hiddenvortices
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
datascience 0.10.6 requires f

In [4]:
# !python ./stylegan2-ada/generate.py generate-images --network="/content/drive/My Drive/test/model.pkl" --seeds=0-10 --outdir="/content/drive/My Drive/test/output"
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [5]:
import sys
import os
# Mount our stylegan repo as a module
sys.path.append('./stylegan2-ada')

# Remove google drive images + video from output folder
path = "/content/drive/My Drive/test/output"
imgs = os.listdir(path)
for img in imgs:
    if (os.path.isdir(img)): continue
    os.remove(f'{path}/{img}')

In [50]:
from google.colab.patches import cv2_imshow
import cv2
import dnnlib.tflib as tflib
import pickle
import os
import dnnlib
import numpy as np
import random

#####
# Generates coaster images based on a trained pickle model file location
# in GDrive as a parameter
#####
def generate_images(network_pkl):
    # TF stuff
    tflib.init_tf()
    print('Loading networks from "%s"...' % network_pkl)
    with dnnlib.util.open_url(network_pkl) as fp:
        _G, _D, Gs = pickle.load(fp)

    Gs_kwargs = {
        'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
        'randomize_noise': False
    }

    # Initial styleGAN parameters
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    label = np.zeros([1] + Gs.input_shapes[1][1:])

    # def getZOSC(*args):
    #     global z
    #     z = np.array(args).reshape(1,512)

    #@markdown #Generation settings
    #@markdown Use a predefined_seed of 0 for random seed
    show_output_image = False #@param {type:"boolean"}
    initial_psi = 1 #@param {type:"number"}
    psi_increment_per_frame = 0.075 #@param {type:"number"}
    zeros = 5     # TODO: This is a quick hacky solution to filenames

    # Begin the generation and output images
    os.chdir("/content/drive/My Drive/test/output/")
    component = Gs.input_shape[1:] # [512]
    z = np.zeros((1,512))

    qlatent1 = np.random.randn(512)[None, :]
    qlatent2 = np.random.randn(512)[None, :]
    linear_space = np.linspace(-0.8, 0.8, 1024)
    interpolation = np.vstack( [(1 - i) * qlatent1 + i * qlatent2 for i in linear_space])
    psi = initial_psi # + psi_increment_per_frame * frame
    Gs_kwargs['truncation_psi'] = psi
    # z = rnd.randn(1, *component) # [minibatch, component]

    noise_rnd = np.random # fix noise
    tflib.set_vars({var: noise_rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]

    for index, value in enumerate(interpolation):
      image = Gs.run(interpolation[index].reshape([1, 512]), None, **Gs_kwargs)[0]

      file_number = str(index).zfill(zeros)
      # Write the image  to g drive and show it.
      cv2.imwrite("output" + file_number + ".jpg", image)
      print("Wrote: output" + file_number + ".jpg")
      if show_output_image is True: cv2_imshow(image)


# Let's run this function!
generate_images("/content/drive/My Drive/test/model.pkl")

Loading networks from "/content/drive/My Drive/test/model.pkl"...
Wrote: output00000.jpg
Wrote: output00001.jpg
Wrote: output00002.jpg
Wrote: output00003.jpg
Wrote: output00004.jpg
Wrote: output00005.jpg
Wrote: output00006.jpg
Wrote: output00007.jpg
Wrote: output00008.jpg
Wrote: output00009.jpg
Wrote: output00010.jpg
Wrote: output00011.jpg
Wrote: output00012.jpg
Wrote: output00013.jpg
Wrote: output00014.jpg
Wrote: output00015.jpg
Wrote: output00016.jpg
Wrote: output00017.jpg
Wrote: output00018.jpg
Wrote: output00019.jpg
Wrote: output00020.jpg
Wrote: output00021.jpg
Wrote: output00022.jpg
Wrote: output00023.jpg
Wrote: output00024.jpg
Wrote: output00025.jpg
Wrote: output00026.jpg
Wrote: output00027.jpg
Wrote: output00028.jpg
Wrote: output00029.jpg
Wrote: output00030.jpg
Wrote: output00031.jpg
Wrote: output00032.jpg
Wrote: output00033.jpg
Wrote: output00034.jpg
Wrote: output00035.jpg
Wrote: output00036.jpg
Wrote: output00037.jpg
Wrote: output00038.jpg
Wrote: output00039.jpg
Wrote: output0

In [51]:
# Convert image sequence to video
!cd "/content/drive/My Drive/test/output/"
# Files must be 01.jpg 02.jpg 03 etc if n<100 image or 001 002 if 100>n<1000 
!ffmpeg -y -pattern_type glob -i "*.jpg" -c:v libx264 -pix_fmt yuv420p -framerate 3 -crf 23 out.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [41]:
# Reverse the output video
!ffmpeg -y -i "/content/drive/My Drive/test/output/out.mp4" -vf reverse out-reversed.mp4


ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib